# TorchVision Tutorial Based Pipeline

Following TORCHVISION OBJECT DETECTION FINETUNING TUTORIAL at https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from pathlib import Path
import os
# Vlad's drive
# repo_path = Path.cwd()/'drive/MyDrive/calcification-detection-project/calcification_detecion/calc-det'
# Joaquin's drive
repo_path = Path.cwd()/'drive/MyDrive/calcification_detection/calc-det/notebooks'
os.chdir(str(repo_path))

thispath = Path.cwd().resolve()
import sys; sys.path.insert(0, str(thispath.parent))

Copy extracted patches used for detection (224 size with stride 100) to the local machine. ***Needs to be done once per machine instantization***. 

~ 3 minutes to complete

In [6]:
# # Vlad's Drive
# # !cp -r /content/drive/MyDrive/calcification-detection-project/calcification_detecion/new_data_rois/data_rois.zip /home/
# # Joaquin's Drive
# !cp -r /content/drive/MyDrive/calcification_detection/data_rois/rois_all_method/data_rois_224.zip /home/data_rois.zip
# !unzip /home/data_rois.zip -d /home
# !mv /home/home/vzalevskyi/projects/data_rois /home/data_rois
# !rm -r /home/home
# !pip install transformers
# !pip install SimpleITK

^C
Archive:  /home/data_rois.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /home/data_rois.zip or
        /home/data_rois.zip.zip, and cannot find /home/data_rois.zip.ZIP, period.
mv: cannot stat '/home/home/vzalevskyi/projects/data_rois': No such file or directory
rm: cannot remove '/home/home': No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 48.4 MB 49 kB/s 


Now let's instantiate the model and the optimizer

And now let's train the model for 10 epochs, evaluating at the end of every epoch.

In [7]:
cfg = {
    'model': {
        'activation': 'LeakyReLU',
        'bloc_act': None,
        'use_middle_activation': True,
        'dropout': 0.4,
        'fc_dims': None,
        'freeze_weights': False,
        'backbone': 'net2',
        'pretrained': True,
        'checkpoint_path': None,
        'anchor_sizes': [12, 14],
        'anchor_ratios': [1.0],
        'num_classes': 2
    },
    'dataset': {
        'extract_patches': False,
        'delete_previous': False,
        'extract_patches_method': 'all',
        'patch_size': 224,
        'stride': 100,
        'min_breast_fraction_roi': 0.7,
        'n_jobs': -1,
        'cropped_imgs': True,
        'ignore_diameter_px': 15,
        'train_neg_to_pos_ratio': None,
        'balancing_seed': 0,
        'normalization': 'min_max',
        'get_lesion_bboxes': True,
        'for_detection_net': True,
        'patch_images_path': '/home/data_rois/',
        'detection_bbox_size':14
    },
    'dataloaders': {
        'train_batch_size': 8,
        'val_batch_size': 16
    },
    'data_aug': {
        'prob': 0
    },
    'training': {
        'criterion': 'BCEWithLogitsLoss',
        'optimizer': 'Adam',
        'optimizer_args': {
            'lr': 0.0001 #, 'momentum': 0.9
        },
        'lr_scheduler': 'StepLR',
        'lr_scheduler_args': {
            'step_size': 3, 'gamma': 0.1
        },
        'n_epochs': 30,
        'best_metric': 'f1_score',
        'resume_training': False,
        'early_stopping': True,
        'early_stopping_args':{
            'min_diff': 0.0001,
            'max_epoch': 3
        },
        'log_iters': 100,
        'max_iters_per_epoch': None
        },
    'experiment_name': '32_resnet_01'
}

import yaml

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)

In [8]:
from pathlib import Path
thispath = Path.cwd().resolve()
import sys; sys.path.insert(0, str(thispath.parent))

import logging
import torch
import torchvision
import yaml

import torch.nn as nn
import torch.optim as optim

from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

import deep_learning.dl_utils as dl_utils
from deep_learning.dataset.dataset import INBreast_Dataset_pytorch
from deep_learning.classification_models.models.base_classifier import CNNClasssifier
from deep_learning.detection_models.train import train_model

logging.basicConfig(level=logging.INFO)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def main():
    # read the configuration file
    config_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
    with open(config_path, "r") as ymlfile:
        cfg = yaml.safe_load(ymlfile)

    # for colab it needs to be specified additionally
    dataset_arguments = cfg['dataset']

    # use the configuration for the dataset
    dataset_arguments = cfg['dataset']
    dataset_arguments['patch_images_path'] = Path(dataset_arguments['patch_images_path'])
    datasets = {
        'train': INBreast_Dataset_pytorch(
            partitions=['train'], neg_to_pos_ratio=dataset_arguments['train_neg_to_pos_ratio'],
            **dataset_arguments),
        'val': INBreast_Dataset_pytorch(
            partitions=['validation'], neg_to_pos_ratio=None, **dataset_arguments)
    }

    # use the configuration for the dataloaders
    def collate_fn(batch):
        return tuple(zip(*batch))
    dataloaders = {
        'val': DataLoader(
            datasets['val'], batch_size=cfg['dataloaders']['val_batch_size'],
            num_workers=4, collate_fn=collate_fn, drop_last=False),
        'train': DataLoader(
            datasets['train'], batch_size=cfg['dataloaders']['train_batch_size'],
            shuffle=True, collate_fn=collate_fn, num_workers=4, drop_last=False)
    }

    # model settings
    if cfg['model']['checkpoint_path'] is not None:
        model_ckpt = torch.load(cfg['model']['checkpoint_path'])
        model = dl_utils.get_model_from_checkpoint(model_ckpt, cfg['model']['freeze_weights'])
    else:
        model = CNNClasssifier(
            activation=getattr(nn, cfg['model']['activation'])(),
            dropout=cfg['model']['dropout'],
            fc_dims=cfg['model']['fc_dims'],
            freeze_weights=cfg['model']['freeze_weights'],
            backbone=cfg['model']['backbone'],
            pretrained=cfg['model']['pretrained'],
        )
        model = model.model

    modules = list(model.children())[:-2]      # delete the last fc layers.
    last_submodule_childs = list(modules[-1][-1].children())
    for i in range(len(last_submodule_childs)-1, -1, -1):
        if hasattr(last_submodule_childs[i], 'out_channels'):
            out_channels = last_submodule_childs[i].out_channels
            break
    model_backbone = nn.Sequential(*modules)
    model_backbone.out_channels = out_channels
    anchor_generator = AnchorGenerator(
        sizes=(cfg['model']['anchor_sizes'],),
        aspect_ratios=(cfg['model']['anchor_ratios'],))

    roi_pooler = torchvision.ops.MultiScaleRoIAlign(
        featmap_names=['0'], output_size=7, sampling_ratio=2)

    # put the pieces together inside a FasterRCNN model
    model = FasterRCNN(
        model_backbone,
        num_classes=cfg['model']['num_classes'],
        rpn_anchor_generator=anchor_generator,
        box_roi_pool=roi_pooler,
        image_mean=[0., 0., 0.],
        image_std=[1., 1., 1.],
    )

    model.to(device)

    # training configs
    optimizer = getattr(optim, cfg['training']['optimizer'])
    optimizer = optimizer(model.parameters(), **cfg['training']['optimizer_args'])

    scheduler = getattr(lr_scheduler, cfg['training']['lr_scheduler'])
    scheduler = scheduler(optimizer, **cfg['training']['lr_scheduler_args'])

    # train the model
    train_model(datasets, dataloaders, model, optimizer, scheduler, cfg)

In [ ]:
# cfg['model']['activation'] = 'LeakyReLU'
# cfg['model']['dropout'] = 0.4
# cfg['model']['fc_dims'] = None
cfg['model']['backbone'] = 'resnet50'
cfg['model']['pretrained'] = True
cfg['model']['checkpoint_path'] = '/content/drive/MyDrive/drive_vlad/deepl_runs/resnet50_05/resnet50_05.pt'
cfg['model']['anchor_sizes'] = [7, 14]
cfg['model']['anchor_ratios'] = [1.0]
cfg['model']['num_classes'] = 2

cfg['dataset']['train_neg_to_pos_ratio'] = None
cfg['dataset']['normalization'] = 'z_score'

cfg['dataloaders']['train_batch_size'] = 6
cfg['dataloaders']['val_batch_size'] = 12

cfg['training']['optimizer'] = 'Adam'
cfg['training']['optimizer_args'] = {'lr': 0.0001}
cfg['training']['lr_scheduler'] = 'StepLR'
cfg['training']['lr_scheduler_args'] = {'step_size': 3, 'gamma': 0.1}
cfg['training']['n_epochs'] = 20
cfg['training']['early_stopping'] = False
cfg['training']['early_stopping_args'] = {'min_diff': 0.0001, 'max_epoch': 3}
cfg['training']['log_iters'] = 100
cfg['training']['best_metric'] = 'AP_IoU_0.50_all'
cfg['training']['max_iters_per_epoch'] = None

cfg['experiment_name'] = 'd_resnet50_06'

cfg_path = str(thispath.parent.parent/'calc-det/deep_learning/detection_models/config.yml')
with open(cfg_path, 'w') as yaml_file:
    yaml.dump(cfg, yaml_file, default_flow_style=False)
main()

/content/drive/MyDrive/calcification_detection/calc-det/deep_learning/dataset/dataset.py:52: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  return_lesions_mask=get_lesion_bboxes, max_lesion_diam_mm=None, use_muscle_mask=False
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

INFO:root:Storing experiment in: /content/drive/MyDrive/calcification_detection/data/deepl_runs/d_resnet50_06


creating index...
index created!
Epoch: [0]  [  0/729]  eta: 1:13:28  lr: 0.000000  loss: 1.6504 (1.6504)  loss_classifier: 0.7561 (0.7561)  loss_box_reg: 0.0024 (0.0024)  loss_objectness: 0.8242 (0.8242)  loss_rpn_box_reg: 0.0677 (0.0677)  time: 6.0468  data: 0.4182  max mem: 9834
Epoch: [0]  [100/729]  eta: 0:11:06  lr: 0.000014  loss: 0.1261 (0.4128)  loss_classifier: 0.0428 (0.1406)  loss_box_reg: 0.0108 (0.0074)  loss_objectness: 0.0564 (0.2317)  loss_rpn_box_reg: 0.0194 (0.0331)  time: 1.0113  data: 0.0191  max mem: 11723
Epoch: [0]  [200/729]  eta: 0:09:07  lr: 0.000028  loss: 0.0864 (0.2716)  loss_classifier: 0.0282 (0.0918)  loss_box_reg: 0.0222 (0.0130)  loss_objectness: 0.0254 (0.1403)  loss_rpn_box_reg: 0.0120 (0.0266)  time: 1.0103  data: 0.0208  max mem: 11723
Epoch: [0]  [300/729]  eta: 0:07:20  lr: 0.000041  loss: 0.0655 (0.2148)  loss_classifier: 0.0204 (0.0717)  loss_box_reg: 0.0222 (0.0188)  loss_objectness: 0.0166 (0.1023)  loss_rpn_box_reg: 0.0078 (0.0220)  time: 1